# Section 2 : Building RecSys Models with Alternating Least Square

### Starting Point:
  
  - Creating a lookup table to track of each item and its description   
  
  
  - Getting RecSys dataset containg "user_info", "item_info" and "item_Quantity"
 

In [936]:
#!pip freeze

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
from pandas.api.types import CategoricalDtype
import warnings
warnings.filterwarnings('ignore')
#pd.options.plotting.backend = 'plotly'

###################################################
import scipy.sparse as sparse
import random
import implicit
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split

########################################################
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from itertools import islice
import pickle


/home/admin1/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
recsys_df = pd.read_csv('Olaleke_Tasks/RecSys_df.csv')
recsys_df.head()

,CardHolder,Material,SalesCount,ItemPrice,net_weight,SubCategorycode,ItemDescription,Class,rpa_wgh2,rpa_wgh2_SubClass,rpa_wgh3,rpa_wgh3_SubClass,vendor
0,10008254,109900,3.0,20.97,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0
1,10011935,109900,1.0,6.49,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0
2,10036221,109900,3.0,19.47,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0
3,10036221,109900,2.0,12.98,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0
4,10036221,109900,4.0,25.96,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0


In [5]:
lenta_bag_df = recsys_df[recsys_df['ItemDescription']== 'Lenta bags']
lenta_bag_df.shape

(1412285, 13)

In [9]:
No_lentabag_df = recsys_df.loc[~recsys_df['ItemDescription'].isin(['Lenta bags'])]
No_lentabag_df.head()

,CardHolder,Material,SalesCount,ItemPrice,net_weight,SubCategorycode,ItemDescription,Class,rpa_wgh2,rpa_wgh2_SubClass,rpa_wgh3,rpa_wgh3_SubClass,vendor
1412285,10008254,439630,1.0,169.99,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0
1412286,14919617,439630,1.0,109.89,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0
1412287,15041309,439630,1.0,169.99,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0
1412288,15041309,439630,1.0,116.99,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0
1412289,15041309,439630,1.0,152.14,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0


In [10]:
No_lentabag_df.shape

(25581761, 13)

In [21]:
#No_lentabag_df.to_csv('No_lentabag.csv',index=False)

In [4]:
No_lentabag_df = pd.read_csv('Olaleke_Tasks/No_lentabag.csv')

In [3]:
No_lentabag_df['ItemDescription'].unique()

array(['Dry dog food up to 800 gr',
       'Dry food for cats from 450 gr to 1500 gr',
       '"Special liquid washing detergent "', ..., 'Hammer drill',
       'Frozen turkey in branded package',
       'Dry food for small breeds (PRO)'], dtype=object)

In [9]:
                                                  #creating a lookup table to track of each item and its description  
                                                  #its a simple pandas dataframe of the unique product ID/product 
                                                  #descriptions available....
#item_lookup = recsys_df[['Material', 'ItemDescription']].drop_duplicates()      #Only get unique item/description pairs
#item_lookup['Material'] = item_lookup['Material'].astype(str)                   #Encode as strings for future lookup ease
#item_lookup = pd.read_csv('Olaleke_Tasks/Item_lookup.csv')
#item_lookup.head()

In [5]:
                                                  #creating a lookup table to track of each item and its description  
                                                  #its a simple pandas dataframe of the unique product ID/product 
                                                  #descriptions available....
#item_lookup_nonbag = No_lentabag_df[['Material', 'ItemDescription']].drop_duplicates()      #Only get unique item/description pairs
#item_lookup_nonbag['Material'] = item_lookup_nonbag['Material'].astype(str)                   #Encode as strings for future lookup ease
item_lookup_nonbag

,Material,ItemDescription
0,439630,Dry dog food up to 800 gr
1,338329,Dry dog food up to 800 gr
2,338283,Dry dog food up to 800 gr
3,338287,Dry dog food up to 800 gr
4,513215,Dry dog food up to 800 gr


In [6]:
item_lookup_nonbag.shape

(58320, 2)

In [7]:
item_lookup_nonbag['Material'] = item_lookup_nonbag['Material'].astype(int)
item_lookup_nonbag.dtypes

Material            int64
ItemDescription    object
dtype: object

In [28]:
#item_lookup_nonbag.to_csv('item_lookup_nonbag.csv',index=False)

In [30]:
#                                   ##groupby CustomerID && Item, then summing quantity values                   
# used_dff = recsys_df[['CardHolder','Material','SalesCount']]
# used_df = used_dff.groupby(['CardHolder','Material']).sum().reset_index()  

#                                   #replacing a sum of 0 with a sum of 1                               
# used_df.SalesCount.loc[used_df.SalesCount==0] = 1
#                                  #get only customers with positive num of quantities
# user_item_info = used_df.query('SalesCount > 0')  
# print(user_item_info.shape)
# user_item_info.head()

In [14]:
                                  ##groupby CustomerID && Item, then summing quantity values                   
used_dff = No_lentabag_df[['CardHolder','Material','SalesCount']]
used_df = used_dff.groupby(['CardHolder','Material']).sum().reset_index()  

                                  #replacing a sum of 0 with a sum of 1                               
used_df.SalesCount.loc[used_df.SalesCount==0] = 1
                                 #get only customers with positive num of quantities
user_item_info = used_df.query('SalesCount > 0')  
print(user_item_info.shape)
user_item_info.head()

(15205494, 3)


,CardHolder,Material,SalesCount
0,10008254,4135,2.0
1,10008254,5383,2.0
2,10008254,5880,2.0
3,10008254,78790,2.0
4,10008254,81107,1.0


In [15]:
#user_item_info.to_csv('user_item_nonbag.csv',index=False)

In [8]:
user_item_info = pd.read_csv('Olaleke_Tasks/user_item_nonbag.csv')
user_item_info.head()

,CardHolder,Material,SalesCount
0,10008254,4135,2.0
1,10008254,5383,2.0
2,10008254,5880,2.0
3,10008254,78790,2.0
4,10008254,81107,1.0


### Creating Rating  Matrix 
 
  - since there are no explicit rating, the purchase quantity can serve as an implicit feedback for us.
  

  - It represent a “confidence” in terms of how strong the user-item was. 
  

  - Items with a larger number of purchases by a customer can carry more weight in the ratings matrix of purchases.
 

In [9]:
                                                                ##creating the sparse rating matrix
CUSTOMERS = list(np.sort(user_item_info['CardHolder'].unique()))   #Get list of unique customers..
ITEMS     = list(user_item_info['Material'].unique())             #Get list of unique items
QUANTITY  =  list(user_item_info['SalesCount'])                       #get all quantities

print("Num of unique customers: ",len(CUSTOMERS))
print("Num of unique Items: ",   len(ITEMS))
print("Num of Item quantities: ",len(QUANTITY))

Num of unique customers:  86497
Num of unique Items:  58320
Num of Item quantities:  15205494


In [9]:
                    #Creating rows values corresponding to unique Customers
rows = user_item_info.CardHolder.astype(CategoricalDtype(categories=CUSTOMERS)).cat.codes
rows

0               0
1               0
2               0
3               0
4               0
            ...  
15205489    86496
15205490    86496
15205491    86496
15205492    86496
15205493    86496
Length: 15205494, dtype: int32

In [10]:
                #Creating columns values corresponding to unique Items
cols = user_item_info.Material.astype(CategoricalDtype(categories=ITEMS)).cat.codes
cols

0               0
1               1
2               2
3               3
4               4
            ...  
15205489    48534
15205490    35770
15205491    41246
15205492    21686
15205493    35162
Length: 15205494, dtype: int32

In [11]:
Rating_matrix = sparse.csr_matrix((QUANTITY, (rows,cols)),
                                     shape=(len(CUSTOMERS),len(ITEMS)))
Rating_matrix

<86497x58320 sparse matrix of type '<class 'numpy.float64'>'
	with 15205494 stored elements in Compressed Sparse Row format>

In [12]:
        ##Checking sparsity of the rating matrix:
Matrix_size = Rating_matrix.shape[0]*Rating_matrix.shape[1]
Num_of_purchase = len(Rating_matrix.nonzero()[0])
sparsity = 100*(1-(Num_of_purchase/Matrix_size))
print("Shape  of the rating matrix: ", Rating_matrix.shape)
print("Size of Rating Matrix: ",Matrix_size)
print("Num of purchase: ",Num_of_purchase)
print("Percentage Sparsity: ", sparsity)

Shape  of the rating matrix:  (86497, 58320)
Size of Rating Matrix:  5044505040
Num of purchase:  15205494
Percentage Sparsity:  99.6985731230432


### Training and Testing set:

- The method of getting training and testing set is as follows;


- Test set is an exact copy of our original rating data.


- The training set, however, will mask a random percentage of user/item interactions and act as if the user never purchased the item (making it a sparse entry with a zero)


- Then during testing, we check in the test set which items were recommended to the user that they ended up actually purchasing.


- If the users frequently ended up purchasing the items most recommended to them by the system, we can conclude the system seems to be working.

In [13]:
                                                ##takes rating matrix and masks a certain 20% 
                                                ##of the user-item interaction
def TrainTest(ratings_matrix,pct_mask=0.2):
    test_set = ratings_matrix.copy()              #making a copy of the rating matrix as text_set
    test_set[test_set!=0]=1                       #storing this as a binary preference matrix 

    train_set = ratings_matrix.copy() 
    nonzero_inds = train_set.nonzero()            #get indices where interaction actually exist
    nonzero_pairs = list(zip(nonzero_inds[0],     #user-item indices into list
                          nonzero_inds[1]))   
    random.seed(0) 
    num_samples = int(np.ceil(pct_mask*len(nonzero_pairs)))  #num of samples to mask
    samples = random.sample(nonzero_pairs, num_samples) 


    user_inds = [i[0] for i in samples]             #Get the user row indices
    item_inds = [i[1] for i in samples]            # Get the item column indices
    train_set[user_inds, item_inds] = 0            # Assign all of the randomly chosen user-item pairs to zero
    train_set.eliminate_zeros()                    # Get rid of zeros in sparse array storage after update to save space
    return train_set, test_set, list(set(user_inds)) # Output the unique list of use

- The function above will return 3 things;


- Train set- {with some samples delibrately masked out},


- Test set - {that has been binarized to 0/1 for purchased/not purchased}


-  and a list of which users had at least one item masked. 


- the performance of the recommender system will be tested on these users only. 

In [14]:
train_set ,test_set,masked_users = TrainTest(Rating_matrix, pct_mask=0.2)

In [15]:
train_set

<86497x58320 sparse matrix of type '<class 'numpy.float64'>'
	with 12164395 stored elements in Compressed Sparse Row format>

In [18]:
len(masked_users)

81755

In [19]:
                #Pickling
with open("masked_users.txt", "wb") as fp:
    pickle.dump(masked_users, fp)
    
sparse.save_npz("ALS_train_set.npz",train_set)
sparse.save_npz("ALS_test_set.npz",test_set)

In [16]:
train_set = sparse.load_npz("ALS_train_set.npz")
test_set = sparse.load_npz("ALS_test_set.npz")
                        #unpickling
with open("masked_users.txt", "rb") as fp:  
        masked_users = pickle.load(fp)

### Implementation of Alternating Least Square [ALS]

- This method  below returns: 


- user_vecs: Array of latent factors for each USERS in the training set (M X K)
    
      
- item_vecs: Array of latent factors for each ITEMS in the training set (K X N) 
     
       
- where K == num of latent factors 

In [852]:
#!pip install implicit

In [17]:
import implicit

In [1]:
#export OPENBLAS_NUM_THREADS=1

In [20]:
alpha = 15
model = implicit.als.AlternatingLeastSquares(factors=50,
                                            regularization = 0.1,
                                            use_gpu = False,  #change to 'True' if connected to GPU 
                                            iterations = 100,
                                            calculate_training_loss = True)


item_user_matrix = (train_set*alpha).astype('double')
model.fit(item_user_matrix.T)
user_vecs = model.user_factors
item_vecs = model.item_factors

In [27]:
# alpha = 15
# user_vecs, item_vecs = implicit.alternating_least_squares((train_set*alpha).astype('double'), 
#                                                           factors=32, 
#                                                           regularization = 0.1, 
#                                                           iterations = 100,
#                                                           use_gpu = False)

In [28]:
#np.save('user_vecs.npy', user_vecs)    #.npy extension is added if not given
#np.save('item_vecs.npy', item_vecs)    #.npy extension is added if not given

In [10]:
user_vecs = np.load('user_vecs.npy')
item_vecs = np.load('item_vecs.npy')

In [11]:
print("Shape of User features: ", user_vecs.shape)
print("Shape of Item features: ", item_vecs.shape)

Shape of User features:  (86497, 50)
Shape of Item features:  (58320, 50)


### Evaluating the ALS RecSys

- The training set has 20% data masked,and this will be used for the evaluation


- The evaluation will be to see **if the order of recommendations given for each user matches the items they ended up purchasing.**


- **Method 1**: using ROC curve; a greater{AUC} means recommending items that end up being purchased near the top of the list of recommended items.

 

In [29]:
                    #creating and AUC_score function
def auc_score(predictions,test):
    fpr,tpr,threshold = metrics.roc_curve(test, predictions)
    auc_score = metrics.auc(fpr, tpr)
    return auc_score

In [30]:
def mean_auc(train_set,test_set,latent_features,masked_users) :
    user_AUC = []
    popularity_AUC = []                     
                                                           #sum of item interactions to find most popular items
    popular_item = np.array(test_set.sum(axis = 0)).reshape(-1)
    item_vectors = latent_features[1]                     #item latent features: matrix_V
    
    for user in masked_users:
        train_row  = train_set[user,:].toarray().reshape(-1)  ##for each user in the training set
        zero_index = np.where(train_row==0)                   ##get the index where interaction has not yet occurred 

                                                              #extract the user latent features... 
        user_vector = latent_features[0][user,:]              #get me each row in the predicted rating matrix--> 
                                                              #user latent features: matrix_U

        pred_rating = user_vector.dot(item_vectors).toarray()[0,zero_index].reshape(-1)  #user predicted ratings based on 

        actual_rating = test_set[user,:].toarray()[0,zero_index].reshape(-1) 
        pop = popular_item[zero_index]                      #item popularity of our chosen item: 
                                                            #to test with our system recommending popular item for every user
        user_AUC.append(auc_score(pred_rating, actual_rating))
        popularity_AUC.append(auc_score(pop,actual_rating))
        AVG_test_AUC = float('%.3f'%np.mean(user_AUC))
        AVG_pop_AUC = float('%.3f'%np.mean(popularity_AUC))

    return AVG_test_AUC,AVG_pop_AUC

In [31]:
                                          ##converting the ALS output to csr_matrix , 
                                          ##...and transposing th eitem vectors:
user_vecs_csr = sparse.csr_matrix(user_vecs) 
item_vecs_csr = sparse.csr_matrix(item_vecs.T) 

latent_features = [user_vecs_csr,item_vecs_csr]
AVG_test_AUC,AVG_pop_AUC  =      mean_auc(train_set,test_set,
                                  latent_features,masked_users)

print("Mean Test AUC:",AVG_test_AUC)        ##AUC based on recommendation on users' interactions
print("Mean Popularity AUC:",AVG_pop_AUC)   ##Auc based on system recommending just the most popular items

Mean Test AUC: 0.872
Mean Popularity AUC: 0.888


- The result above is not satisfactory enough, 


- because recommendation based on the  most popular items performs better than the usr_item interaction

### ALS Recoemmdation Example 

In [143]:
item_lookup_nonbag.head()

,Material,ItemDescription
0,439630,Dry dog food up to 800 gr
1,338329,Dry dog food up to 800 gr
2,338283,Dry dog food up to 800 gr
3,338287,Dry dog food up to 800 gr
4,513215,Dry dog food up to 800 gr


In [99]:
customers_arr = np.array(CUSTOMERS)     #Array of unique customer IDs from the ratings matrix
items_arr =  np.array(ITEMS)            #Array of unique product IDs from the ratings matrix

                                        #get the items that have already been purchased by
                                        #...a specific customer in the training set
def get_item_already_bought(customer_id,train_set,customers_list,items_list,item_lookup):
    customers_index = np.where(customers_list == customer_id)[0][0]    #get the row index  of the customer_ID 
    customer_item_index = train_set[customers_index,:].nonzero()[1]    #get column indices of purchased items by the customer 
    Item_id = items_list[customer_item_index] 
    get_item = item_lookup.loc[item_lookup['Material'].isin(Item_id)]
    get_item =  get_item.reset_index(drop=True)                                                          #return :A list of item IDs and item descriptions 
    return get_item                                             #..for a particular customer that were already purchased in the training set     

In [121]:
# item_lookup['Material'] = item_lookup['Material'].astype(int)
# item_lookup['Material'].dtype

item_lookup = item_lookup_nonbag 

In [148]:
                             #function to recommend top items for all users... 
def get_recommended_items(customer_id,train_set=train_set,user_vecs=user_vecs, 
                          item_vecs=item_vecs,customer_list=customers_arr,
                          item_list = items_arr,item_lookup=item_lookup, num_items = 10):
    
    
    customers_index = np.where(customer_list == customer_id)[0][0] 
    customer_rating = train_set[customers_index,:].toarray()          #get customer rating from the rating_matrix(train_set)
    customer_rating = customer_rating.reshape(-1) + 1                 #adding 1 to every values; items not purchased yet(0 before), will not become 1
    customer_rating[customer_rating > 1] = 0                          #make all items already purchased  to be 0
    recommend_vec = user_vecs[customers_index,:].dot(item_vecs.T)     #Get dot product of user vector and all item vectors
                                                                      #making the recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(recommend_vec.reshape(-1,1))[:,0] 

    recommend_vector = customer_rating*rec_vector_scaled              #item already purchased have their recommendation multiplied by 0
    item_index = np.argsort(recommend_vector)[::-1][:num_items]       #Sorting the indices of the recommended_items into order 
    recommend_list = []
    for i in item_index[:num_items]:
        code = item_list[i]                                            #appening item description to list
        recommend_list.append([code,item_lookup.ItemDescription.loc[item_lookup.Material == code].iloc[0]]) 
        
        #get_item = item_lookup.loc[item_lookup['Material'].isin(Item_id)]
    codes = [item[0] for item in recommend_list]
    descriptions = [item[1] for item in recommend_list]
    final_frame = pd.DataFrame({'Material': codes, 'ItemDescription': descriptions})   #Create a dataframe 
    recommended = final_frame[['Material', 'ItemDescription']]                         #Switch order of columns around
    
                        ##get items already bought
    customers_index = np.where(customer_list == customer_id)[0][0]    #get the row index  of the customer_ID 
    customer_item_index = train_set[customers_index,:].nonzero()[1]    #get column indices of purchased items by the customer 
    Item_id = item_list[customer_item_index] 
    already_bought = item_lookup.loc[item_lookup['Material'].isin(Item_id)]
    already_bought =  already_bought.reset_index(drop=True)
                                                                #return :A list of item IDs and item descriptions 

           
    compare_output = pd.concat([already_bought[:num_items]['ItemDescription'], 
                                recommended[:num_items]['ItemDescription']],
                                ignore_index=True, axis=1)
    compare_output = compare_output.rename(columns={0:"Already_Purchased_Items",
                                                    1:"Recommended_Items(ALS)"})
    
    print("Recommendation for user_id: {}".format(customer_id))
    return already_bought,recommended,compare_output

In [157]:
customers_arr[-10:]  #view the 1st 5 customers_id

array([83326249, 83326911, 83327390, 83329160, 83329286, 83329684,
       83329792, 83332008, 83332870, 83333056])

In [158]:
get_recommended_items(83326249)[2]  

Recommendation for user_id: 83326249


,Already_Purchased_Items,Recommended_Items(ALS)
0,"""Table and drinking still water ""","""Table and drinking still water """
1,NaN,Baby water
2,NaN,Baby juice
3,NaN,Baby water
4,NaN,"Kefir, bifidus for children under 3 years"
5,NaN,Chocolate eggs
6,NaN,Cream yoghurt for children under 3 years
7,NaN,"""Table and drinking still water """
8,NaN,Baby water
9,NaN,Chocolate eggs


In [151]:
get_recommended_items(83329160)[2]

Recommendation for user_id: 83329160


,Already_Purchased_Items,Recommended_Items(ALS)
0,Butter packing 125 gr - 300 gr,"""Medicinal mineral water """
1,Pants,Bananas
2,"""Medicinal mineral water ""","""Medicinal mineral water """
3,"""Cables for mobile phones ""","Baguette, own production"
4,"""Caviar imitation in branded package """,Premium and imported water
5,NaN,Cow sterilized milk (UHT) <1 l
6,NaN,Bulb onion
7,NaN,"""Table and drinking still water """
8,NaN,"Wheat bread, own production"
9,NaN,A loaf of wheat in sliced.


In [150]:
get_recommended_items(83327390)[2]

Recommendation for user_id: 83327390


,Already_Purchased_Items,Recommended_Items(ALS)
0,Oatmeal cookies,Puff pastry treats
1,Bulk grapes,Puff pastry treats
2,"""Bulk apples """,Puff pastry treats
3,"Juices Tetra Pack 0,6-1,4 л. Other flavours",Puff pastry treats
4,"""Table and drinking still water """,Puff pastry treats
5,Frankfurters in branded package,A loaf of wheat in sliced.
6,Flavoured black tea bags,Puff pastry treats
7,"Fruit drink 0,6-1,4 l","""Sweet dough treats with fruit and berry filli..."
8,"""Fresh semi-finished products from cut poultry...",Other pastries
9,A loaf of wheat in sliced.,Puff pastry treats


In [153]:
get_recommended_items(83329286)[2]

Recommendation for user_id: 83329286


,Already_Purchased_Items,Recommended_Items(ALS)
0,Cow pasteurized milk <1 l,Bananas
1,Imported beer in can,"Baguette, own production"
2,Other oriental cuisine products,"""Table and drinking still water """
3,Table and drinking water without gas 4l or more,Bulb onion
4,Contraceptives,Watermelons
5,"""Other fresh mushrooms """,Cow sterilized milk (UHT) <1 l
6,NaN,"Wheat bread, own production"
7,NaN,Premium and imported water
8,NaN,Cow sterilized milk (UHT) <1 l
9,NaN,Cola-based lemonade


In [152]:
get_recommended_items(10049978)[2]

Recommendation for user_id: 10049978


,Already_Purchased_Items,Recommended_Items(ALS)
0,Bananas,Premium and imported water
1,Condensed milk,Premium and imported water
2,Premium and imported water,Premium and imported water
3,Cashew,Premium and imported water
4,Grape seed oil,Premium and imported water
5,Olive oil Pomace,Cola-based lemonade
6,"""Russian brandy """,Premium and imported water
7,Milk shakes cold storage,"""Table and drinking still water """
8,NaN,"""Table and drinking still water """
9,NaN,"""Medicinal mineral water """


In [127]:
get_recommended_items(10011587)[2]

Recommendation for user_id: 10011587


,Already_Purchased_Items,Recommended_Items(ALS)
0,"""Sparkling white wine, brut, dry, semi-dry """,Table and drinking water without gas 4l or more
1,Cow pasteurized milk <1 l,Imported beer in can
2,Oatmeal cookies,Table and drinking water without gas 4l or more
3,Oatmeal cookies,Gingerbread
4,NaN,Gingerbread
5,NaN,"""Medicinal mineral water """
6,NaN,"""Medicinal mineral water """
7,NaN,Oatmeal cookies
8,NaN,Oatmeal cookies
9,NaN,Imported beer in bottle


In [128]:
get_recommended_items(10011935)[2]  

Recommendation for user_id: 10011935


,Already_Purchased_Items,Recommended_Items(ALS)
0,"Baking foil, sleeve",Spoonable yoghurts cold storage
1,"""Low-fat sterilized cream (below 15%) """,One flower type bouquets
2,"Bottled juice 0,6-1,4 l","""Edam, Gouda, Tilsiter cheese sold by weight """
3,"Chicken eggs, more than 15 eggs",Fresh whole and cut turkey own product
4,Oatmeal cookies,Spoonable yoghurts cold storage
5,Oatmeal cookies,Classic mashmallow
6,"""Plastic construction toys """,Fresh whole and cut turkey own product
7,Modelling sets,"""Natural curd, ? 5% fat """
8,Modelling sets,"Curd rolls, snacks cold storage"
9,Modelling sets,Spoonable yoghurts cold storage


In [129]:
get_recommended_items(10036221)[2]

Recommendation for user_id: 10036221


,Already_Purchased_Items,Recommended_Items(ALS)
0,Cotton wool,"""Frozen one vegetable type mix, packaged """
1,Cotton wool,Individual portion food for dogs (pouch)
2,Cotton wool,Individual portion food for dogs (pouch)
3,"Washing detergent - capsules, tablets","Biokefir, kefir drinks"
4,"Washing detergent - capsules, tablets","""Frozen one vegetable type mix, packaged """
5,Fresh cut chicken in branded package,Cream yoghurt for children under 3 years
6,Fresh cut chicken in branded package,"Biokefir, kefir drinks"
7,Fresh cut chicken in branded package,Cream yoghurt for children under 3 years
8,"Cut salmon, service",Ready-to-eat baby cereals
9,"Cut salmon, service",Fresh whole and cut turkey own product


In [26]:
#                         ##get item already bought by a user '10049978'...
# already_purchased2 = get_item_already_bought(10011587,train_set,customers_arr,items_arr,item_lookup)

# rec_items2 = get_recommended_items(10011587, train_set,user_vecs, item_vecs, 
#                       customers_arr,items_arr, item_lookup,num_items = 5)

# pd.DataFrame({'Purchased_Item':already_purchased2['ItemDescription'].values,
#         'Recommended_items':rec_items2[:4]['ItemDescription'].values}) 

In [15]:
#                         ##get item already bought by a user '10049978'...
# already_purchased3 = get_item_already_bought(10011935,train_set,customers_arr,items_arr,item_lookup)

# rec_items3 = get_recommended_items(10011935, train_set,user_vecs, item_vecs, 
#                       customers_arr,items_arr, item_lookup,num_items = 10)

# pd.DataFrame({'Purchased_Item':already_purchased3[:10]['ItemDescription'].values,
#         'Recommended_items':rec_items3['ItemDescription'].values}) 